Imports

In [1]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:1", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 12

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:1'
)
# Load the SAE weights using safetensors
from safetensors import safe_open
with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
    w_dec = st.get_tensor("W_dec").to('cuda:1')



In [2]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(1)
torch.cuda.current_device()


1

In [12]:
#sae.W_dec == w_dec

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(w_dec[471,:], sae.W_dec).topk(100).indices

tensor([ 3629,   475,  6443,   787, 14794,  5972,  5503, 15488,  4383,  4722,
         8050,  4991,  7091, 12567,  1729,  8691, 12185, 16042, 13945, 13265,
        11450,  5611,  9251, 14709,  6738,  5659,  9041,  8431,  8211, 14557,
        11135,  6775,  1687,  8728, 10550,  9459,  6943, 14992,  9952, 15132,
         1004, 13303, 15616,  8370, 13180, 15194, 12101, 10275,   889, 14433,
        11472,  9166,  1254,  1276, 12955, 10202, 13693,  7128,  9954, 12760,
         1977,  4701, 10394, 10754,  7344,  8475, 15455,  1913,  2356,  5581,
         3519,   172, 12890, 13044,  9313, 11299, 10402, 10261,  5454,  6346,
         5624,  2969,  9894,  9198,  1568, 10725,  9632,  4256,    92, 15403,
         2624, 14645,  3363,  1053,  1419,  1258, 15607, 15804,   923, 15685],
       device='cuda:1')

Parameters

In [47]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629    
scale = 55.5 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [66]:
full_res = []
#v = range(0,10)
for ismpl in v:


    # prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive, 
    # somewhat positive, neutral, somewhat negative, 
    # very negative? """ + ismpl + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'


    prompt = """<start_of_turn>user\n The president anounced new tariffs but at the same time promised tax cuts.<end_of_turn>\n<start_of_turn>model\n 
    How would that affect the economy of the country? """

    
    #prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

    # prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
    positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
    with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
        # vector steering section
        # for feature_i in topics:
        vector = sae.W_dec[[feature]]
        vector = vector / vector.norm()
        vector = vector * scale
        # setting vector into layer 2
        # for position in positions:
        #   model.model.layers[2].output[0][:, position] = vector
            
        # get final output saved
        out = model.generator.output.save()
        # capture at layer 6 or 12
        resid = model.model.layers[12].output.save()
        
    features = sae.encode(resid[0])
    summed_activations = features.abs().sum(dim=1) # Sort by max activations
    summed_activations = features
    # top_activations_indices = summed_activations.topk(200).indices # Get indices of top 20    
    top_activations_indices = summed_activations.indices # Get indices of top 20    

    
    for i, l in enumerate(model.tokenizer.batch_decode(out)):
        s = repr(l.partition(prompt)[2].partition("<eos>")[0])
        print(prompt + "|"+ s + "|" + str(top_activations_indices))
        # print(str(top_activations_indices))


<start_of_turn>user
 The president anounced new tariffs but at the same time promised tax cuts.<end_of_turn>
<start_of_turn>model
 
    How would that affect the economy of the country? |"\n\nThe president's announcement of new tariffs would likely lead to a decrease in imports, as businesses would be less likely to purchase goods from abroad if they become more expensive. This would have a ripple effect on the economy, as it would lead to a decrease in demand for other goods and services, which would in turn lead to a decrease in employment and production.\n\nOn the other hand, the promise of tax cuts would likely lead to an increase in spending, as consumers would be more likely to buy goods and services if they are more affordable. This would lead to a boost in the economy, as it would increase demand for goods and services. \n\nOverall, the impact of the president's announcement would be complex and depends on a number of factors, including the specific details of the tariffs and t

## summed activations across tokens, finance features

In [67]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[True, False, False, False, False, False, False, False, False, False, False]

## Get features for each token

In [78]:
summed_activations


tensor([[[1.9184, 1.9771, 0.0000,  ..., 3.8246, 0.0000, 2.4047],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
       device='cuda:1', grad_fn=<ReluBackward0>)